In [1]:
import codecs
import networkx as nx
import os
import pickle
from networkx.drawing.nx_pydot import write_dot
from os import listdir
from os.path import isfile, join
from utils import printProgressBar

In [2]:
vertex_labels = pickle.load(open('label_vertex.p', 'rb'))
edge_labels = pickle.load(open('label_edge.p', 'rb'))
family_labels = pickle.load(open('label_family.p', 'rb'))

# without joining graphs that have the same members

In [3]:
result_file_name = 'result_gSpan-s100_l3.txt'
params = result_file_name.split('-')[1].replace('.txt', '')
temp_out_dir = 'graphs_from_gSpan'
H = nx.DiGraph()
with codecs.open(result_file_name, 'r', encoding='utf-8') as f:
    for line in f:
        if line[0] == 't':
            result_number = line.split()[2]
            vertex_label_indices = list()
        elif line[0] == 'v':
            vertex_infos = line.split()
            H.add_node(vertex_infos[1], label=vertex_labels[int(vertex_infos[2])])
        elif line[0] == 'e':
            edge_infos = line.split()
            full_label = edge_labels[int(edge_infos[3])]
            edge_label = full_label.split('_')[0] + '_' + full_label.split('_')[-1]
            if 'NA' in full_label:
                vertices = sorted([edge_infos[1], edge_infos[2]])
                H.add_edge(vertices[0], vertices[1], label=edge_label, dir='none', style='dashed', color='gray')
            elif 'indirect' in full_label:
                vertices = sorted([edge_infos[1], edge_infos[2]])
                H.add_edge(vertices[0], vertices[1], label=edge_label, dir='none', style='dashed')
            else:
                H.add_edge(edge_infos[1], edge_infos[2], label=edge_label)
        elif line[0] == 'w':
            members = line.replace('\r\n', '').replace(' ', '').replace('[', '').replace(']', '').split(':')[1]
        elif line[0] == '-':
            f_out = codecs.open(join(temp_out_dir, params + '_' + str(result_number).rjust(4, '0') + '.members.txt'), 'w')
            member_list = members.split(',')
            member_list = list(map(int, member_list))
            member_list.sort()
            for member in member_list:
                #f_out.write('F' + str(member).rjust(5, '0') + '\n')
                f_out.write(family_labels[member] + '\n')
            f_out.close()
            write_dot(H, join(temp_out_dir, params + '_' + str(result_number).rjust(4, '0') + '.dot'))
            H.clear()

# with joining graphs that have the same members

In [ ]:
# because of 'a+' writing options, the directory should be empty
joined_result_dir = 'joined_results'
result_file_name = 'result_gSpan-s50_l3.txt'
params = result_file_name.split('-')[1].replace('.txt', '')
out_names = list()
with codecs.open(result_file_name, 'r', encoding='utf-8') as f:
    res = ''
    members = ''
    for line in f:
        res += line
        if line[0] == '-':
            f_out = codecs.open(join(joined_result_dir, params + '_' + str(out_name).rjust(4, '0') + '.txt'), 'a+', encoding='utf-8')
            f_out.write(res)
            f_out.close()
            res = ''
        elif line[0] == 'w':
            members = line.replace('\r\n', '').replace(' ', '').replace('[', '').replace(']', '').split(':')[1]
            try:
                out_name = out_names.index(members)
            except:
                out_name = len(out_names)
                out_names.append(members)

In [ ]:
input_files = [f for f in listdir(joined_result_dir) if isfile(join(joined_result_dir, f))]
temp_out_dir = 'graphs_from_gSpan'
for file_counter, input_file in enumerate(input_files):
    H = nx.DiGraph()
    with codecs.open(join(joined_result_dir, input_file), 'r', encoding='utf-8') as f:
        for line in f:
            if line[0] == 't':
                result_number = line.split()[2]
                vertex_label_indices = list()
            elif line[0] == 'v':
                vertex_infos = line.split()
                H.add_node(vertex_infos[1], label=vertex_labels[int(vertex_infos[2])])
            elif line[0] == 'e':
                edge_infos = line.split()
                full_label = edge_labels[int(edge_infos[3])]
                edge_label = full_label.split('_')[0] + '_' + full_label.split('_')[-1]
                if 'NA' in full_label:
                    vertices = sorted([edge_infos[1], edge_infos[2]])
                    H.add_edge(vertices[0], vertices[1], label=edge_label, dir='none', style='dashed', color='gray')
                elif 'indirect' in full_label:
                    vertices = sorted([edge_infos[1], edge_infos[2]])
                    H.add_edge(vertices[0], vertices[1], label=edge_label, dir='none', style='dashed')
                else:
                    H.add_edge(edge_infos[1], edge_infos[2], label=edge_label)
            elif line[0] == 'w':
                members = line.replace('\r\n', '').replace(' ', '').replace('[', '').replace(']', '').split(':')[1]
            elif line[0] == '-':
                f_out = codecs.open(join(temp_out_dir, input_file.replace('.txt', '.members.txt')), 'w')
                member_list = members.split(',')
                member_list = list(map(int, member_list))
                member_list.sort()
                for member in member_list:
                    f_out.write('F' + str(member).rjust(5, '0') + '\n')
                f_out.close()
    write_dot(H, join(temp_out_dir, input_file.replace('.txt', '.dot')))
    printProgressBar(file_counter, len(input_files), prefix = 'Progress:', suffix = 'complete', length = 50, decimals = 2)
    H.clear()